In [13]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\test-20230326T155708Z-001\test\COVID\COVID-19 (8).jpeg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 423ms/step
Predicted class: COVID


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121, InceptionV3

# Define ensemble model using DenseNet121 and InceptionV3
def ensemble_model(input_shape=(224, 224, 3), num_classes=6):
    # Load pre-trained DenseNet121 model
    densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in densenet_model.layers:
        layer.trainable = False

    # Load pre-trained InceptionV3 model
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in inception_model.layers:
        layer.trainable = False

    # Define input layer
    input_layer = layers.Input(shape=input_shape)

    # Extract features using DenseNet121
    densenet_features = densenet_model(input_layer)
    densenet_features = layers.GlobalAveragePooling2D()(densenet_features)

    # Extract features using InceptionV3
    inception_features = inception_model(input_layer)
    inception_features = layers.GlobalAveragePooling2D()(inception_features)

    # Concatenate features
    merged_features = layers.Concatenate()([densenet_features, inception_features])

    # Add a fully connected layer
    merged_features = layers.Dense(512, activation='relu')(merged_features)

    # Output layer
    output = layers.Dense(num_classes, activation='softmax')(merged_features)

    # Create model
    model = Model(inputs=input_layer, outputs=output)

    return model

# Define data directories
train_dir = r'D:\new smote\train-20230326T152931Z-001\train'
val_dir = r'D:\new smote\val-20230326T152503Z-001\val'
test_dir = r'D:\new smote\test-20230326T155708Z-001\test'
image_size = (224, 224)
batch_size = 32

# Create data generators for training, validation, and testing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)

# Load ensemble model
model = ensemble_model(input_shape=image_size + (3,), num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=200,
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the model
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)


Found 16050 images belonging to 6 classes.
Found 4104 images belonging to 6 classes.
Epoch 1/10


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 3580s 16s/step - accuracy: 0.5432 - loss: 1.6773 - val_accuracy: 0.7393 - val_loss: 0.7135
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1954s 10s/step - accuracy: 0.7592 - loss: 0.6655 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


200/200 ━━━━━━━━━━━━━━━━━━━━ 1673s 8s/step - accuracy: 0.7828 - loss: 0.6067 - val_accuracy: 0.7476 - val_loss: 0.6722
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1245s 6s/step - accuracy: 0.7915 - loss: 0.5851 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1454s 7s/step - accuracy: 0.7838 - loss: 0.5791 - val_accuracy: 0.8009 - val_loss: 0.5380
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 575s 3s/step - accuracy: 0.8123 - loss: 0.4984 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1940s 10s/step - accuracy: 0.8165 - loss: 0.5048 - val_accuracy: 0.7953 - val_loss: 0.5390
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 630s 3s/step - accuracy: 0.7978 - loss: 0.5426 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 678s 3s/step - accuracy: 0.8383 - loss: 0.4660 - val_accuracy: 0.7968 - val_loss: 0.5678
Epoch 10/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 598s 3s/step - accuracy: 0.8257 - loss

In [2]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions on the test set
y_pred_probabilities = model.predict(test_generator)
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert true labels to class indices
y_true = test_generator.classes

# Generate classification report
report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


129/129 ━━━━━━━━━━━━━━━━━━━━ 385s 3s/step
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.16      0.18      0.17       685
       COVID       0.16      0.18      0.17       685
      NORMAL       0.18      0.17      0.18       685
   PNEUMONIA       0.16      0.18      0.17       685
PNEUMOTHORAX       0.17      0.11      0.13       685
TUBERCULOSIS       0.17      0.18      0.18       685

    accuracy                           0.17      4110
   macro avg       0.17      0.17      0.17      4110
weighted avg       0.17      0.17      0.17      4110

Confusion Matrix:
[[123 118 100 141  90 113]
 [134 126 116 108  63 138]
 [117 126 116 131  79 116]
 [134 123 110 122  74 122]
 [126 138  99 129  76 117]
 [133 138  91 129  70 124]]


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121, InceptionV3

# Define ensemble model using DenseNet121 and InceptionV3
def ensemble_model(input_shape=(224, 224, 3), num_classes=6):
    # Load pre-trained DenseNet121 model
    densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in densenet_model.layers:
        layer.trainable = False

    # Load pre-trained InceptionV3 model
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in inception_model.layers:
        layer.trainable = False

    # Define input layer
    input_layer = layers.Input(shape=input_shape)

    # Extract features using DenseNet121
    densenet_features = densenet_model(input_layer)
    densenet_features = layers.GlobalAveragePooling2D()(densenet_features)

    # Extract features using InceptionV3
    inception_features = inception_model(input_layer)
    inception_features = layers.GlobalAveragePooling2D()(inception_features)

    # Concatenate features
    merged_features = layers.Concatenate()([densenet_features, inception_features])

    # Add a fully connected layer
    merged_features = layers.Dense(512, activation='relu')(merged_features)

    # Output layer
    output = layers.Dense(num_classes, activation='softmax')(merged_features)

    # Create model
    model = Model(inputs=input_layer, outputs=output)

    return model

# Define data directories
train_dir = r'D:\new smote\train-20230326T152931Z-001\train'
val_dir = r'D:\new smote\val-20230326T152503Z-001\val'
test_dir = r'D:\new smote\test-20230326T155708Z-001\test'
image_size = (224, 224)
batch_size = 32

# Create data generators for training, validation, and testing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)

# Load ensemble model
model = ensemble_model(input_shape=image_size + (3,), num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=200,
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the model
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)


Found 16050 images belonging to 6 classes.
Found 4104 images belonging to 6 classes.
Epoch 1/20


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 1391s 7s/step - accuracy: 0.5479 - loss: 1.6399 - val_accuracy: 0.7191 - val_loss: 0.7464
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 875s 4s/step - accuracy: 0.7508 - loss: 0.6683 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


200/200 ━━━━━━━━━━━━━━━━━━━━ 1206s 6s/step - accuracy: 0.7955 - loss: 0.5630 - val_accuracy: 0.7943 - val_loss: 0.5794
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1192s 6s/step - accuracy: 0.7931 - loss: 0.5685 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1087s 5s/step - accuracy: 0.7952 - loss: 0.5566 - val_accuracy: 0.8095 - val_loss: 0.5238
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 302s 2s/step - accuracy: 0.8268 - loss: 0.4839 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 961s 5s/step - accuracy: 0.8088 - loss: 0.5244 - val_accuracy: 0.7956 - val_loss: 0.5786
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 600s 3s/step - accuracy: 0.8110 - loss: 0.5221 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 673s 3s/step - accuracy: 0.8234 - loss: 0.4847 - val_accuracy: 0.8197 - val_loss: 0.4925
Epoch 10/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 596s 3s/step - accuracy: 0.8321 - loss: 

In [4]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions on the test set
y_pred_probabilities = model.predict(test_generator)
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert true labels to class indices
y_true = test_generator.classes

# Generate classification report
report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


129/129 ━━━━━━━━━━━━━━━━━━━━ 393s 3s/step
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.18      0.13      0.16       685
       COVID       0.18      0.20      0.19       685
      NORMAL       0.17      0.18      0.18       685
   PNEUMONIA       0.15      0.14      0.14       685
PNEUMOTHORAX       0.17      0.16      0.17       685
TUBERCULOSIS       0.16      0.19      0.17       685

    accuracy                           0.17      4110
   macro avg       0.17      0.17      0.17      4110
weighted avg       0.17      0.17      0.17      4110

Confusion Matrix:
[[ 92 115 113 124 108 133]
 [ 79 137 134 106  98 131]
 [ 76 120 125 104 125 135]
 [ 72 121 126  95 118 153]
 [ 96 130 110 101 112 136]
 [ 85 120 124 122 105 129]]


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121, InceptionV3

# Define ensemble model using DenseNet121 and InceptionV3
def ensemble_model(input_shape=(224, 224, 3), num_classes=6):
    # Load pre-trained DenseNet121 model
    densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in densenet_model.layers:
        layer.trainable = False

    # Load pre-trained InceptionV3 model
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in inception_model.layers:
        layer.trainable = False

    # Define input layer
    input_layer = layers.Input(shape=input_shape)

    # Extract features using DenseNet121
    densenet_features = densenet_model(input_layer)
    densenet_features = layers.GlobalAveragePooling2D()(densenet_features)

    # Extract features using InceptionV3
    inception_features = inception_model(input_layer)
    inception_features = layers.GlobalAveragePooling2D()(inception_features)

    # Concatenate features
    merged_features = layers.Concatenate()([densenet_features, inception_features])

    # Add a fully connected layer
    merged_features = layers.Dense(512, activation='relu')(merged_features)

    # Output layer
    output = layers.Dense(num_classes, activation='softmax')(merged_features)

    # Create model
    model = Model(inputs=input_layer, outputs=output)

    return model

# Define data directories
train_dir = r'D:\new smote\train-20230326T152931Z-001\train'
val_dir = r'D:\new smote\val-20230326T152503Z-001\val'
test_dir = r'D:\new smote\test-20230326T155708Z-001\test'
image_size = (224, 224)
batch_size = 32

# Create data generators for training, validation, and testing
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)

# Load ensemble model
model = ensemble_model(input_shape=image_size + (3,), num_classes=num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=200,
    epochs=30,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the model
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)


Found 16050 images belonging to 6 classes.
Found 4104 images belonging to 6 classes.
Epoch 1/30


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 1023s 5s/step - accuracy: 0.5381 - loss: 1.6906 - val_accuracy: 0.7222 - val_loss: 0.7240
Epoch 2/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 614s 3s/step - accuracy: 0.7593 - loss: 0.6744 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/30


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


200/200 ━━━━━━━━━━━━━━━━━━━━ 697s 3s/step - accuracy: 0.7748 - loss: 0.6125 - val_accuracy: 0.7364 - val_loss: 0.7132
Epoch 4/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 617s 3s/step - accuracy: 0.7916 - loss: 0.5703 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1001s 5s/step - accuracy: 0.7912 - loss: 0.5630 - val_accuracy: 0.7018 - val_loss: 0.7822
Epoch 6/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - accuracy: 0.7898 - loss: 0.5731 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1000s 5s/step - accuracy: 0.8226 - loss: 0.4982 - val_accuracy: 0.8185 - val_loss: 0.5025
Epoch 8/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 613s 3s/step - accuracy: 0.8266 - loss: 0.4847 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 693s 3s/step - accuracy: 0.8228 - loss: 0.4915 - val_accuracy: 0.7963 - val_loss: 0.5445
Epoch 10/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 611s 3s/step - accuracy: 0.8409 - loss: 0

In [6]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions on the test set
y_pred_probabilities = model.predict(test_generator)
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Convert true labels to class indices
y_true = test_generator.classes

# Generate classification report
report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


129/129 ━━━━━━━━━━━━━━━━━━━━ 396s 3s/step
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.16      0.17      0.17       685
       COVID       0.17      0.15      0.16       685
      NORMAL       0.16      0.14      0.15       685
   PNEUMONIA       0.19      0.18      0.18       685
PNEUMOTHORAX       0.19      0.24      0.21       685
TUBERCULOSIS       0.15      0.15      0.15       685

    accuracy                           0.17      4110
   macro avg       0.17      0.17      0.17      4110
weighted avg       0.17      0.17      0.17      4110

Confusion Matrix:
[[116  94 111 115 141 108]
 [111 101  88 105 150 130]
 [139  97  97 111 123 118]
 [112 114  89 122 144 104]
 [117  89  97  99 161 122]
 [123 106 122 105 128 101]]
